In [ ]:
# define libraries to use
import pandas as pd
import numpy as np
from pathlib import Path
import requests,json
import matplotlib.pyplot as plt
import math

# variables that need initialization

# dict of dataframes

# CRRS
crssDf = {}
# NY state vehicle registrations
#nydmvDf = None

# Main Table
farsMainTbl = "accident"
farsKey = "CASENUM"
crssMainTbl = "accident"
crssKey = "CASENUM"

def bulkreadcsv2df(myFolder, printProgress = False, lowMemory = True):
    """read each csv file in a folder into a df, each df added to as a separate dictionary entry with index a filename"""
    p = Path(myFolder)
    resDct = {}
    fileList = list(p.glob("*.CSV")) + list(p.glob("*.csv"))
    # sort file list based on stem of filename in lowercase
    fileList = sorted(fileList,key=lambda ppath: str(ppath.stem).lower())
    for myFile in fileList:
        dfName = myFile.stem.lower()
        if printProgress:
            print("Reading ",dfName)
        resDct[dfName] = pd.read_csv(str(myFile),encoding = 'unicode_escape', low_memory=lowMemory)

    return resDct



In [ ]:
#crssDf = bulkreadcsv2df("data/FARS2020NationalCSV/", lowMemory= False)
crssDf = bulkreadcsv2df("data/CRSS2020CSV/", lowMemory= False)
#crssDf = bulkreadcsv2df("data/CRSS2016CSV/", lowMemory= False)
#nydmvDf = pd.read_csv("data/nydmv/nydmv.csv",encoding = "unicode_escape", low_memory=False)

#print("FARS Tables = ",list(crssDf.keys()))
print("CRSS Tables = ",list(crssDf.keys()))

In [ ]:
print("                               C    R    S    S                                                           ")
for idx in crssDf.keys():
    print("==========================================================================================================")
    print("=                                                                                                        =")
    print("=",idx)
    crssDf[idx].info(verbose=True,show_counts=True)
    print("=                                                                                                        =")
    print("==========================================================================================================")


# CRSS Table Cleanup to reduce to columns of interest and remove redundant data

## Create Categorical variables based on data

Pandas categorical data type not used so as to keep the original category codes

In [ ]:
# crssCategories will hold the categorical mappings for categorical colums

crssCategories = {}

crssCategoricalColumns = {
    "accident" : [
        ["STRATUM","STRATUMNAME"],
        ["REGION","REGIONNAME"],
        ["MONTH","MONTHNAME"],
        ["DAY_WEEK","DAY_WEEKNAME"],
        ["URBANICITY","URBANICITYNAME"],
        ["HARM_EV","HARM_EVNAME"],
        ["MAN_COLL","MAN_COLLNAME"],
        ["ALCOHOL","ALCOHOLNAME"],
        ["MAX_SEV","MAX_SEVNAME"],
        ["LGT_COND","LGT_CONDNAME"],
        ["WEATHER","WEATHERNAME"],
        ["LGTCON_IM","LGTCON_IMNAME"],
        ["WEATHR_IM","WEATHR_IMNAME"],
        ["MAXSEV_IM","MAXSEV_IMNAME"],
        ["ALCHL_IM","ALCHL_IMNAME"],
    ],
    "vehicle" : [
        ["NUMOCCS","NUMOCCSNAME"],
        ["MAKE","MAKENAME"],
        ["MAK_MOD","MAK_MODNAME"],
        ["BODY_TYP","BODY_TYPNAME"],
        ["BDYTYP_IM","BDYTYP_IMNAME"],
        ["TOW_VEH","TOW_VEHNAME"],
        ["TRAV_SP","TRAV_SPNAME"],
        ["ROLLOVER","ROLLOVERNAME"],
        ["M_HARM","M_HARMNAME"],
        ["DR_PRES","DR_PRESNAME"],
        #["L_STATUS","L_STATUSNAME"],
        #["L_TYPE","L_TYPENAME"],
        #["PREV_ACC","PREV_ACCNAME"],
        #["PREV_DWI","PREV_DWINAME"],
        #["PREV_SPD","PREV_SPDNAME"],
        #["PREV_OTH","PREV_OTHNAME"],
        ["SPEEDREL","SPEEDRELNAME"],
        ["VSURCOND","VSURCONDNAME"],
        ["P_CRASH2","P_CRASH2NAME"],
        ["ACC_TYPE","ACC_TYPENAME"],
        ["VEH_ALCH","VEH_ALCHNAME"],
        ["V_ALCH_IM","V_ALCH_IMNAME"],
        ["VPICMAKE","VPICMAKENAME"],
        ["VPICMODEL","VPICMODELNAME"],
        ["VPICBODYCLASS","VPICBODYCLASSNAME"],
    ],
    "person" : [
        ["SEX","SEXNAME"],
        ["SEX_IM","SEX_IMNAME"],
        ["PER_TYP","PER_TYPNAME"],
        ["INJ_SEV","INJ_SEVNAME"],
        ["INJSEV_IM","INJSEV_IMNAME"],
        ["SEAT_POS","SEAT_POSNAME"],
        ["REST_USE","REST_USENAME"],
        ["AIR_BAG","AIR_BAGNAME"],
        ["EJECTION","EJECTIONNAME"],
        ["EJECT_IM","EJECT_IMNAME"],
        ["LOCATION","LOCATIONNAME"],
        ["DRINKING","DRINKINGNAME"],
        ["PERALCH_IM","PERALCH_IMNAME"],
    ],
    "parkwork" : [
		["PNUMOCCS","PNUMOCCSNAME"],
		["PTYPE","PTYPENAME"],
		["PMAKE","PMAKENAME"],
		["PMAK_MOD","PBODYTYP"],
		["PTRAILER","PTRAILERNAME"],
		["PM_HARM","PM_HARMNAME"],
    ],
    "pbtype" : [
        ["PBPTYPE","PBPTYPENAME"],
        ["PBAGE","PBAGENAME"],
        ["PBSEX","PBSEXNAME"],
    ],
    "vevent" : [
        ["AOI1","AOI1NAME"],
        ["AOI2","AOI2NAME"],
    ],
    "crashrf" : [
        ["CRASHRF","CRASHRFNAME"],
    ],
    "driverrf" : [
        ["DRIVERRF","DRIVERRFNAME"],
    ],
    "distract" : [
        ["DRDISTRACT","DRDISTRACTNAME"],
    ],
    "drimpair" : [
        ["DRIMPAIR", "DRIMPAIRNAME"],
    ],
    "factor" : [
        ["VEHICLECC","VEHICLECCNAME"],
    ],
    "vision" : [
        ["VISION", "VISIONNAME"],
    ],
    "nmcrash" : [
        ["NMCC","NMCCNAME"],
    ],
    "vpicdecode" : [
        ["VehicleTypeId","VehicleType"],
		["ManufacturerFullNameId","ManufacturerFullName"],
		["MakeId","Make"],
		["ModelId","Model"],
		["BodyClassId","BodyClass"],
        ["GrossVehicleWeightRatingFromId","GrossVehicleWeightRatingFrom"],
        ["GrossVehicleWeightRatingToId","GrossVehicleWeightRatingTo"],
        # safety features
		["ForwardCollisionWarningId","ForwardCollisionWarning"],
		["DynamicBrakeSupportId","DynamicBrakeSupport"],
		["CrashImminentBrakingID","CrashImminentBraking"],
		["PedestrianAutoEmergencyBrakingId","PedestrianAutoEmergencyBraking"],
		["BlindSpotWarningId","BlindSpotWarning"],
		["BlindSpotInterventionId","BlindSpotIntervention"],
		["LaneDepartureWarningId","LaneDepartureWarning"],
		["LaneKeepingAssistanceId","LaneKeepingAssistance"],
		["LaneCenteringAssistanceId","LaneCenteringAssistance"],
		["BackupCameraId","BackupCamera"],
		["RearCrossTrafficAlertId","RearCrossTrafficAlert"],
		["RearAutomaticEmergencyBrakingId","RearAutomaticEmergencyBraking"],
		["ParkAssistId","ParkAssist"],
		["DaytimeRunningLightId","DaytimeRunningLight"],
		["HeadlampLightSourceId","HeadlampLightSource"],
		["SemiAutoHeadlampBeamSwitchingId","SemiAutoHeadlampBeamSwitching"],
		["AdaptiveDrivingBeamId","AdaptiveDrivingBeam"],
		["AdaptiveCruiseControlId","AdaptiveCruiseControl"],
		["AntilockBrakeSystemId","AntilockBrakeSystem"],
		["ElectronicStabilityControlId","ElectronicStabilityControl"],
		["TPMSId","TPMS"],
		["TractionControlId","TractionControl"],
		["AutoPedestrianAlertingSoundId","AutoPedestrianAlertingSound"],
    ]
}

for dataFile in crssCategoricalColumns.keys():
    print(dataFile, "======================================")
    for catCol in crssCategoricalColumns[dataFile]:
        print(catCol, end=",")
        crssCategories[catCol[0]] = dict(np.array(crssDf[dataFile][catCol].drop_duplicates()))
    print()



In [ ]:
# FARS non-categorical Columns

crssNumericalColumns = {
    "accident" : [
		"CASENUM",
		"PSU",
		"PJ",
		"PSU_VAR",
		"VE_TOTAL",
		"VE_FORMS",
		"PVH_INVL",
		"PEDS",
		#"PERSONS",
		"PERMVIT",
		"PERNOTMVIT",
		#"DAY",
		"YEAR",
		"HOUR",
		"MINUTE",
		#"FATALS",
		"NUM_INJ",
		"NO_INJ_IM",
		"WEIGHT",
    ],
    "vehicle" : [
		"CASENUM",
		"VEH_NO",
		"MOD_YEAR",
		"MDLYR_IM",
		"VIN",
		#"DEATHS",
		"WEIGHT",
    ],
    "person" : [
		"CASENUM",
		"VEH_NO",
		"PER_NO",
		"STR_VEH",
		"AGE",
		"AGE_IM",
		"WEIGHT",
    ],
    "parkwork" : [
		"CASENUM",
		"VEH_NO",
		"PBODYTYPNAME",
		"PMODYEAR",
		"PVIN",
		#"PDEATHS",
		"WEIGHT",
    ],
    "pbtype" : [
		"CASENUM",
		"VEH_NO",
		"PER_NO",
		"WEIGHT",
    ],
    "vevent" : [
		"CASENUM",
		"VEH_NO",
		#"VEVENTNUM",
		"VNumber1",
		"VNumber2",
		"VNumber2NAME",
		"WEIGHT",
    ],
    "crashrf" : [
		"CASENUM",
		"WEIGHT",
    ],
    "driverrf" : [
		"CASENUM",
		"VEH_NO",
		"WEIGHT",
    ],
    "distract" : [
		"CASENUM",
		"VEH_NO",
		"WEIGHT",
    ],
    "drimpair" : [
		"CASENUM",
		"VEH_NO",
		"WEIGHT",
    ],
    "factor" : [
		"CASENUM",
		"VEH_NO",
		"WEIGHT",
    ],
    "vision" : [
		"CASENUM",
		"VEH_NO",
		"WEIGHT",
    ],
    "nmcrash" : [
		"CASENUM",
		"VEH_NO",
		"PER_NO",
		"WEIGHT",
    ],
	"vpicdecode" : [
		"CASENUM",
		"VEH_NO",
		"VehicleDescriptor",
		"VINDecodeError",
		"ModelYear",
		"Series",
		"Trim",
		"Series2",
		"Trim2",
		"CurbWeightLB",
	]
} 

# Drop all columns except chosen ones
for dataFile in crssNumericalColumns.keys():
	print(dataFile, "======================================")
	# all columns
	columnSet = set(crssDf[dataFile].columns)
	for catCol in crssCategoricalColumns[dataFile]:
		# remove categorical index column so that it is preserved (not dropped)
		columnSet.remove(catCol[0])
	for catCol in crssNumericalColumns[dataFile]:
		# remove numerical column we want to preserve (not dropped)
		columnSet.remove(catCol)
	# remove unwanted columns
	crssDf[dataFile].drop(columns = columnSet,inplace = True)
	print(crssDf[dataFile].columns) 

In [ ]:
#pd.DataFrame(crssCategories["HARM_EV"],columns=["id","name"]).to_csv("HARM_EV.csv")

# add simplified HARM_EV category HARM_EV_SIMP, and simplified PHARM_EV_SIMP
crssCategories["HARM_EV_SIMP"] = dict(
    [
        [1,"Animal"],
	    [2,"Building"],
	    [3,"Fixed object"],
	    [4,"Jackknife"],
	    [5,"Motor Vehicle In-Transport"],
	    [6,"Non-Collision"],
	    [7,"Non-Motorist on Personal Conveyance"],
	    [8,"Object not-fixed"],
	    [9,"Parked Motor Vehicle"],
	    [10,"Road Surface"],
	    [11,"Pedalcyclist"],
	    [12,"Pedestrian"],
	    [13,"Railway Vehicle"],
	    [14,"Rollover/Overturn"],
	    [15,"Snow Bank"],
	    [16,"Traffic Barrier"],
	    [17,"Working Motor Vehicle"],
	    [99,"Unknown"]
    ]
)
#crssCategories["PHARM_EV_SIMP"] = crssCategories["HARM_EV_SIMP"].copy()

# HARM_EV mapping to HARM_EV_SIMP
harmEvSimpMap = {
	1 : 14,
	2 : 6,
	3 : 6,
	5 : 6,
	6 : 6,
	7 : 6,
	8 : 12,
	9 : 11,
	10 : 13,
	11 : 1,
	12 : 5,
	14 : 9,
	15 : 7,
	16 : 8,
	17 : 8,
	18 : 8,
	19 : 2,
	20 : 16,
	21 : 16,
	23 : 16,
	24 : 16,
	25 : 16,
	26 : 16,
	30 : 3,
	31 : 3,
	32 : 3,
	33 : 3,
	34 : 3,
	35 : 3,
	38 : 3,
	39 : 3,
	40 : 3,
	41 : 3,
	42 : 3,
	43 : 3,
	44 : 10,
	45 : 17,
	46 : 16,
	48 : 15,
	49 : 1,
	50 : 16,
	51 : 4,
	52 : 16,
	53 : 3,
	54 : 5,
	55 : 5,
	57 : 3,
	58 : 10,
	59 : 3,
	72 : 8,
	73 : 8,
	74 : 13,
	91 : 8,
	93 : 3,
	98 : 99,
	99 : 99,
}

# new columns based on mappings
crssDf["accident"]["HARM_EV_SIMP"] = crssDf["accident"].apply(lambda row: harmEvSimpMap[row["HARM_EV"]],axis=1)
crssDf["parkwork"]["PM_HARM_SIMP"] = crssDf["parkwork"].apply(lambda row: harmEvSimpMap[row["PM_HARM"]],axis=1)

In [ ]:
crssDf["accident"].describe(include="all")

In [ ]:
crssDf["accident"].info(verbose=True,show_counts=True)

In [ ]:
crssDf["vehicle"].describe(include="all")

In [ ]:
crssDf["vehicle"].info(verbose=True,show_counts=True)

In [ ]:
crssDf["vpicdecode"].describe(include="all")

In [ ]:
crssDf["vpicdecode"].info(verbose=True,show_counts=True)

In [ ]:
totalWeightedAccidentsOrig = round(crssDf["accident"]["WEIGHT"].sum())
totalWeightedVehiclesOrig  = round(crssDf["vehicle"]["WEIGHT"].sum())
totalWeightedFatalsOrig    = round(crssDf["accident"][crssDf["accident"]["MAXSEV_IM"]==4]["WEIGHT"].sum())
print("Total Accidents (weighted) of original dataset       =",totalWeightedAccidentsOrig)
print("Total Fatal Accidents (weighted) of original dataset =",totalWeightedFatalsOrig)
print("Total Vehicles (weighted) of original dataset        =",totalWeightedVehiclesOrig)

In [ ]:
# verify if all CASENUM combinations are present
accStCaseSet = set(crssDf["accident"]["CASENUM"].dropna().unique())
vehStCaseSet = set(crssDf["vehicle"]["CASENUM"].dropna().unique())
vpicStCaseSet = set(crssDf["vpicdecode"]["CASENUM"].dropna().unique())

print(len(accStCaseSet),len(vehStCaseSet),len(vpicStCaseSet))
print("Set diff 'accident' - 'vehicle' \t=\t",len(accStCaseSet.copy().difference(vehStCaseSet)))
print("Set diff 'accident' - 'vpicdecode' \t=\t",len(accStCaseSet.copy().difference(vpicStCaseSet)))
print("Set diff 'vehicle' - 'vpicdecode' \t=\t",len(vehStCaseSet.copy().difference(vpicStCaseSet)))
print("Set diff 'vpicdecode' - 'vehicle' \t=\t",len(vpicStCaseSet.copy().difference(vehStCaseSet)))
print(len(accStCaseSet),len(vehStCaseSet),len(vpicStCaseSet))

# there are cases without vpicdecode entries, all cases have vehicle entries

# find cases without vpic decode entry
tempDf = crssDf["accident"][~crssDf["accident"]["CASENUM"].isin(vpicStCaseSet)]
accCountWithoutVPIC = len(tempDf)

# find all vehicles wihtout vpic decode entry
temp2Df = crssDf["vehicle"][~crssDf["vehicle"]["CASENUM"].isin(vpicStCaseSet)]
vehBodyTypeWithoutVPIC = temp2Df.apply(lambda row: crssCategories["BODY_TYP"][row["BODY_TYP"]],axis=1).value_counts()
vehMakeWithoutVPIC = temp2Df.apply(lambda row: crssCategories["MAKE"][row["MAKE"]],axis=1).value_counts()
vehModelWithoutVPIC = temp2Df.apply(lambda row: crssCategories["MAK_MOD"][row["MAK_MOD"]],axis=1).value_counts()
#vehModelYearWithoutVPIC = temp2Df["MOD_YEAR"].value_counts()
vehModelYearWithoutVPIC = temp2Df["MDLYR_IM"].value_counts()
vehCountWithoutVPIC = len(temp2Df)
print("total accidents wiht vehicles wihtout VPIC =\t",accCountWithoutVPIC,accCountWithoutVPIC/len(crssDf["accident"])*100,"%")
print("total vehicles wihtout VPIC =\t\t\t",vehCountWithoutVPIC,vehCountWithoutVPIC/len(crssDf["vehicle"])*100,"%")
print("Most frequent Body Type without VPIC =\t",vehBodyTypeWithoutVPIC.head(1).index[0],"\tfreq =",vehBodyTypeWithoutVPIC.head(1)[0],vehBodyTypeWithoutVPIC.head(1)[0]/vehCountWithoutVPIC*100,"%")
print("Most frequent Make without VPIC =\t",vehMakeWithoutVPIC.head(1).index[0],"\t\tfreq =",vehMakeWithoutVPIC.head(1)[0],vehMakeWithoutVPIC.head(1)[0]/vehCountWithoutVPIC*100,"%")
print("Most frequent Model without VPIC =\t",vehModelWithoutVPIC.head(1).index[0],"\t\tfreq =",vehModelWithoutVPIC.head(1)[0],vehModelWithoutVPIC.head(1)[0]/vehCountWithoutVPIC*100,"%")
print("Most frequent Model Year without VPIC =\t",vehModelYearWithoutVPIC.head(1).index[0],"\t\t\tfreq =",vehModelYearWithoutVPIC.head(1).values[0],vehModelYearWithoutVPIC.head(1).values[0]/vehCountWithoutVPIC*100,"%")

#tempDf = None
#temp2Df = None

print(crssDf["vehicle"]["MDLYR_IM"].drop_duplicates().sort_values().values)
print(crssDf["vehicle"]["MOD_YEAR"].drop_duplicates().sort_values().values)
print(crssDf["vpicdecode"]["ModelYear"].drop_duplicates().sort_values().values)
print(vehModelWithoutVPIC)

In [ ]:
print("Total (Weighted) accidents without VPIC = ",round(tempDf["WEIGHT"].sum()),tempDf["WEIGHT"].sum()/totalWeightedAccidentsOrig*100,"%")
print("Total (Weighted) vehicles  without VPIC = ",round(temp2Df["WEIGHT"].sum()),temp2Df["WEIGHT"].sum()/totalWeightedVehiclesOrig*100,"%")

#temp2Df[["MAK_MOD","MDLYR_IM","WEIGHT"]].groupby(["MAK_MOD","MDLYR_IM"]).sum()
topModelsWithoutVPIC = temp2Df[["MAK_MOD","WEIGHT"]].groupby(["MAK_MOD"]).sum().sort_values(by="WEIGHT",ascending=False)
topModelsWithoutVPIC["Model"] = topModelsWithoutVPIC.index
topModelsWithoutVPIC["Model"] = topModelsWithoutVPIC.apply(lambda row: crssCategories["MAK_MOD"][row["Model"]],axis=1)
topModelsWithoutVPIC["Pct"] = topModelsWithoutVPIC["WEIGHT"] / temp2Df["WEIGHT"].sum() * 100
topModelsWithoutVPIC.head(10)


In [ ]:
#tempDf = crssDf["accident"][~crssDf["accident"]["CASENUM"].isin(vpicStCaseSet)]
#temp2Df = crssDf["vehicle"][~crssDf["vehicle"]["CASENUM"].isin(vpicStCaseSet)]
#temp2Df[temp2Df["MOD_YEAR"]==2019]["VIN"].value_counts()

In [ ]:
# remove accidents and vehicles without vpic decode
# vehicles without vpic record
crssDf["vehicle"] = pd.merge(crssDf["vehicle"],crssDf["vpicdecode"], on = ["CASENUM","VEH_NO"])[crssDf["vehicle"].columns]
# remove accidents that have vehicles without vpic records
crssDf["accident"] = crssDf["accident"][crssDf["accident"]["CASENUM"].isin(set(crssDf["vehicle"]["CASENUM"].dropna().unique()))].copy()


In [ ]:
crssDf["accident"].describe(include="all")

# CRSS Data Verification

In [ ]:
totalAcc = len(crssDf["accident"])

In [ ]:
accStatesDf = pd.DataFrame(crssDf["accident"]["REGION"].value_counts().sort_values(ascending=False))
accStatesDf.columns = ["TotalAcc"]
accStatesDf["REGION"] = accStatesDf.index.values
accStatesDf["REGION"] = accStatesDf.apply(lambda row: crssCategories["REGION"][row["REGION"]],axis=1)
#accStatesDf.loc["Others"] = [crssDf["accident"]["REGION"].count() - accStatesDf["TotalAcc"].sum(),"Others"]
accStatesDf.set_index("REGION",inplace=True)
accStatesDf

In [ ]:

def func(pct):
    absolute = int(pct/100.*crssDf["accident"]["REGION"].count())
    return "{:.1f}%\n({:d})".format(pct, absolute)

#autopct = "%1.1f%%" if not using function

_ = accStatesDf.plot.pie(y="TotalAcc",figsize=(8,8),legend = False, title = "Traffic Accident Sample Distribution per U.S. Region (2020)", \
    ylabel = "", autopct=lambda pct: func(pct), pctdistance = 0.9, rotatelabels = False)



In [ ]:
accStatesWghtDf = crssDf["accident"][["REGION","WEIGHT"]].groupby("REGION").sum().sort_values(by="WEIGHT",ascending=False)
accStatesWghtDf.columns = ["TotalAcc"]
accStatesWghtDf["REGION"] = accStatesWghtDf.index.values
accStatesWghtDf["REGION"] = accStatesWghtDf.apply(lambda row: crssCategories["REGION"][row["REGION"]],axis=1)
##accStatesDf.loc["Others"] = [crssDf["accident"]["REGION"].count() - accStatesDf["TotalAcc"].sum(),"Others"]
accStatesWghtDf.set_index("REGION",inplace=True)
totalWeightedAccidents = accStatesWghtDf["TotalAcc"].sum()
accStatesWghtDf

In [ ]:

def func(pct):
    absolute = int(pct/100.*totalWeightedAccidents)
    return "{:.1f}%\n({:,})".format(pct, absolute)

#autopct = "%1.1f%%" if not using function

_ = accStatesWghtDf.plot.pie(y="TotalAcc",figsize=(8,8),legend = False, title = "Traffic Accident Weighted Sample Distribution per U.S. Region (2020)", \
    ylabel = "", autopct=lambda pct: func(pct), pctdistance = 0.9, rotatelabels = False)



In [ ]:
#accTotalsDf = pd.DataFrame(dict(crssDf["accident"][["VE_TOTAL","VE_FORMS","PVH_INVL","PERSONS","PERMVIT","PERNOTMVIT","PEDS","FATALS"]].sum()),index = ["Total"])
accTotalsDf = pd.DataFrame(dict(crssDf["accident"][["VE_TOTAL","VE_FORMS","PVH_INVL","PERMVIT","PERNOTMVIT","PEDS","NUM_INJ"]].sum()),index = ["Total"])
accTotalsDf["Accidents"] = len(crssDf["accident"])
accTotalsDf["Occupants in Parked and Work Vehicles"] = accTotalsDf["PERNOTMVIT"] - accTotalsDf["PEDS"]
accTotalsDf = accTotalsDf.reindex(columns=['Accidents', 'VE_TOTAL', "VE_FORMS", 'PVH_INVL', 'PERMVIT', 'PERNOTMVIT', "Occupants in Parked and Work Vehicles",'PEDS','NUM_INJ'])
accTotalsDf.columns = ['Total Accidents', 'Total Vehicles', "Vehicles in Transport",'Parked and Working Vehicles', 'Occupants in Vehicles in Transport', 'Persons Not in Motor Vehicles in Transport', "Occupants in Parked and Work Vehicles", 'Person not in Vehicles','Number of Injuries']
accTotalsDf

In [ ]:
accTotalsSer = accTotalsDf.loc["Total"]
ax = accTotalsSer.plot.barh(title = "U.S. 2020 Sample Accident Totals")

In [ ]:
crssMaxSevDf = pd.DataFrame(crssDf["accident"]["MAX_SEV"].value_counts().sort_values())
crssMaxSevDf.columns = ["TotalAccidents"]
crssMaxSevDf["MAX_SEV"] = crssMaxSevDf.index.values
crssMaxSevDf["MAX_SEV"] = crssMaxSevDf.apply(lambda row: crssCategories["MAX_SEV"][row["MAX_SEV"]],axis=1)
crssMaxSevDf.set_index("MAX_SEV",inplace=True)
crssMaxSevDf

In [ ]:
crssMaxSevDftmp = crssMaxSevDf[crssMaxSevDf["TotalAccidents"]/crssMaxSevDf["TotalAccidents"].sum() >= 0.02].copy()
crssMaxSevDftmp.loc["Other/Unknown"] = [crssMaxSevDf["TotalAccidents"].sum() - crssMaxSevDftmp["TotalAccidents"].sum()]
crssMaxSevDftmp["TotalAccidents"].plot.pie(legend = False, ylabel = "", autopct="%1.1f%%", pctdistance = 0.7, rotatelabels = False, title = "Sampled Accidents Injury Types")

In [ ]:
accTotalsDf.loc["Total"][["Vehicles in Transport",'Parked and Working Vehicles']].plot.pie(legend = False, title = "Vehicles in Transport vs Parked and Working Vehicles", \
    ylabel = "", autopct="%1.1f%%", pctdistance = 0.7, rotatelabels = False)

In [ ]:
_ = accTotalsDf.loc["Total"][['Occupants in Vehicles in Transport',"Occupants in Parked and Work Vehicles"]].plot.pie(legend = False, title = "Occupants in Vehhicles in Transport vs Parked and Working Vehicles", \
    ylabel = "", autopct="%1.1f%%", pctdistance = 0.7, rotatelabels = False)

In [ ]:
# accidents by month
accMonthDf = pd.DataFrame(crssDf["accident"]["MONTH"].value_counts()).sort_index()
accMonthDf.columns = ["TotalAcc"]
accMonthDf["MONTH"] = accMonthDf.index.values
accMonthDf["MONTH"] = accMonthDf.apply(lambda row: crssCategories["MONTH"][row["MONTH"]],axis=1)
#accMonthDf.loc["Others"] = [crssDf["accident"]["STATE"].count() - accMonthDf["TotalAcc"].sum(),"Others"]
accMonthDf.set_index("MONTH",inplace=True)
accMonthDf

In [ ]:
accMonthDf["TotalAcc"].plot.bar(title = "U.S. 2020 Sampled Traffic Accident per Month",legend = False, ylabel = "Total Falta Accidents")

In [ ]:
# accidents by day of the week
accDayWeekDf = pd.DataFrame(crssDf["accident"]["DAY_WEEK"].value_counts()).sort_index()
accDayWeekDf.columns = ["TotalAcc"]
accDayWeekDf["DAY_WEEK"] = accDayWeekDf.index.values
accDayWeekDf["DAY_WEEK"] = accDayWeekDf.apply(lambda row: crssCategories["DAY_WEEK"][row["DAY_WEEK"]],axis=1)
#accMonthDf.loc["Others"] = [crssDf["accident"]["STATE"].count() - accMonthDf["TotalAcc"].sum(),"Others"]
accDayWeekDf.set_index("DAY_WEEK",inplace=True)
accDayWeekDf

In [ ]:
accDayWeekDf.plot.bar(title = "U.S. 2020 Sampled Traffic Accident per Day of the Week",legend = False, ylabel = "Total Falta Accidents", \
    xlabel = "Day of the Week")

In [ ]:
# accidents and weather events
accWeatherDf = pd.DataFrame(crssDf["accident"]["WEATHER"].value_counts().sort_values(ascending=False))
accWeatherDf.columns = ["TotalAccidents"]
totalWeatherAcc = accWeatherDf["TotalAccidents"].sum()
accWeatherDf["WEATHER"] = accWeatherDf.index.values
accWeatherDf["WEATHER"] = accWeatherDf.apply(lambda row: crssCategories["WEATHER"][row["WEATHER"]],axis=1)
accWeatherDf.set_index("WEATHER",inplace=True)
# combined unknown and not reported rows
accWeatherDf.loc["Unknown"] = accWeatherDf.loc["Not Reported"] + accWeatherDf.loc["Reported as Unknown"]
accWeatherDf.drop(index=["Reported as Unknown","Not Reported"],inplace=True)
accWeatherDf

In [ ]:
accWeatherDf = accWeatherDf[accWeatherDf["TotalAccidents"] > 2000]
accWeatherDf.loc["Other"] = [totalWeatherAcc - accWeatherDf["TotalAccidents"].sum()]
_ = accWeatherDf["TotalAccidents"].plot.pie(legend = False, title = "Total Accidents under reported Weather conditions", \
    ylabel = "", autopct="%1.1f%%", pctdistance = 0.7, rotatelabels = False)

In [ ]:
# accidents and light condition
accLightDf = pd.DataFrame(crssDf["accident"]["LGT_COND"].value_counts().sort_values(ascending=False))
accLightDf.columns = ["TotalAccidents"]
totalAcc = accLightDf["TotalAccidents"].sum()
accLightDf["LGT_COND"] = accLightDf.index.values
accLightDf["LGT_COND"] = accLightDf.apply(lambda row: crssCategories["LGT_COND"][row["LGT_COND"]],axis=1)
accLightDf.set_index("LGT_COND",inplace=True)
# combined unknown and not reported rows
#accLightDf.loc["Unknown"] = accWeatherDf.loc["Not Reported"] + accWeatherDf.loc["Reported as Unknown"]
#accLightDf.drop(index=["Reported as Unknown","Not Reported"],inplace=True)
accLightDf

In [ ]:
accLightDf = accLightDf[accLightDf["TotalAccidents"] > 900]
accLightDf.loc["Other"] = [totalAcc - accLightDf["TotalAccidents"].sum()]
_ = accLightDf["TotalAccidents"].plot.pie(legend = False, title = "Total Sampled Accidents under different light conditions", \
    ylabel = "", autopct="%1.1f%%", pctdistance = 0.7, rotatelabels = False)

In [ ]:
# accidents harmul events
accHarmEvDf = pd.DataFrame(crssDf["accident"]["HARM_EV_SIMP"].value_counts().sort_values(ascending=False))
accHarmEvDf.columns = ["TotalAccidents"]
accHarmEvDf["HARM_EV_SIMP"] = accHarmEvDf.index.values
accHarmEvDf["HARM_EV_SIMP"] = accHarmEvDf.apply(lambda row: crssCategories["HARM_EV_SIMP"][row["HARM_EV_SIMP"]],axis=1)
#accMonthDf.loc["Others"] = [crssDf["accident"]["STATE"].count() - accMonthDf["TotalAcc"].sum(),"Others"]
accHarmEvDf.set_index("HARM_EV_SIMP",inplace=True)
totalAcc = accHarmEvDf["TotalAccidents"].sum()
accHarmEvDf

In [ ]:
accHarmEvDf = accHarmEvDf[accHarmEvDf["TotalAccidents"] > 500].copy()
accHarmEvDf.loc["Other"] = [totalAcc - accHarmEvDf["TotalAccidents"].sum()]
_ = accHarmEvDf["TotalAccidents"].plot.pie(legend = False, title = "Sampled First injury or damage producing event of the crash", \
    ylabel = "", autopct="%1.1f%%", pctdistance = 0.7, rotatelabels = False)

In [ ]:
# for Events including Motort Vehicle in Transport orientation of montor vehicles
accManCollDf = pd.DataFrame(crssDf["accident"]["MAN_COLL"].value_counts().sort_values(ascending=False))
accManCollDf.columns = ["TotalAccidents"]
accManCollDf["MAN_COLL"] = accManCollDf.index.values
accManCollDf["MAN_COLL"] = accManCollDf.apply(lambda row: crssCategories["MAN_COLL"][row["MAN_COLL"]],axis=1)
#accMonthDf.loc["Others"] = [crssDf["accident"]["STATE"].count() - accMonthDf["TotalAcc"].sum(),"Others"]
accManCollDf.set_index("MAN_COLL",inplace=True)
accManCollDf

In [ ]:
accManCollDf.drop(index=["The First Harmful Event was Not a Collision with a Motor Vehicle In Transport"],inplace=True)
totMVinTransportAcc = accManCollDf["TotalAccidents"].sum()
accManCollDf

In [ ]:
# keep evetns that make at least 3% of total
accManCollDf = accManCollDf[accManCollDf["TotalAccidents"]/totMVinTransportAcc >= 0.03]
accManCollDf.loc["Other"] = [totMVinTransportAcc - accManCollDf["TotalAccidents"].sum()]
_ = accManCollDf["TotalAccidents"].plot.pie(legend = False, title = "Orientation of two MVIT involved in the crash", \
    ylabel = "", autopct="%1.1f%%", pctdistance = 0.7, rotatelabels = False)

In [ ]:
# Vehicle Make 
# accidents harmul events
vehModDf = pd.DataFrame(crssDf["vehicle"]["MAKE"].value_counts().sort_values(ascending=False)).head(20)
vehModDf.columns = ["TotalVehicles"]
vehModDf["MAKE"] = vehModDf.index.values
vehModDf["MAKE"] = vehModDf.apply(lambda row: crssCategories["MAKE"][row["MAKE"]],axis=1)
vehModDf.set_index("MAKE",inplace=True)
totalVeh = len(crssDf["vehicle"])
vehModDf.loc["Others"] = [totalVeh - vehModDf["TotalVehicles"].sum()]
vehModDf


In [ ]:
#accTotalsSer = vehModDf.loc["TotalVehicles"]
ax = vehModDf.plot.barh(title = "Make of Vehicles Present in Fatal Accidents")

In [ ]:
# Vehicle Model Year
vehYrDf = pd.DataFrame(crssDf["vehicle"]["MOD_YEAR"].value_counts())
vehYrDf.columns = ["TotalVehicles"]
vehYrDf["MOD_YEAR"] = vehYrDf.index.values
#vehYrDf["MAKE"] = vehYrDf.apply(lambda row: crssCategories["MAKE"][row["MAKE"]],axis=1)
#vehYrDf.set_index("MOD_YEAR",inplace=True)
totalVeh = len(crssDf["vehicle"])
#vehYrDf.loc["Others"] = [totalVeh - vehYrDf["TotalVehicles"].sum()]
#vehYrDf.loc["Unknown"] = [vehYrDf.loc[9998]["TotalVehicles"] + vehYrDf.loc[9999]["TotalVehicles"],np.NaN]
#vehYrDf.drop(index=[9998,9999],inplace=True)
vehYrDf

In [ ]:
_ = crssDf["vehicle"][crssDf["vehicle"]["MOD_YEAR"] < 9998]["MOD_YEAR"].plot.hist(bins=40,title="Vehicle Model Year Distribution")

In [ ]:
#ax = vehYrDf.boxplot(column="MOD_YEAR")
ax = crssDf["vehicle"][crssDf["vehicle"]["MOD_YEAR"] < 9998].boxplot(column="MOD_YEAR")
ax.set_title("Vehicle Model Year")

In [ ]:
# Vehicle Body class
vehBodyDf = pd.DataFrame(crssDf["vehicle"]["VPICBODYCLASS"].value_counts())
vehBodyDf.columns = ["TotalVehicles"]
vehBodyDf["VPICBODYCLASS"] = vehBodyDf.index.values
vehBodyDf["VPICBODYCLASS"] = vehBodyDf.apply(lambda row: crssCategories["VPICBODYCLASS"][row["VPICBODYCLASS"]],axis=1)
vehBodyDf.set_index("VPICBODYCLASS",inplace=True)
vehBodyDf

In [ ]:
# keep evetns that make at least 3% of total
vehBodyDf = vehBodyDf[vehBodyDf["TotalVehicles"]/totalVeh >= 0.034].copy()
vehBodyDf.loc["Other"] = totalVeh - vehBodyDf["TotalVehicles"].sum()
_ = vehBodyDf["TotalVehicles"].plot.pie(legend = False, title = "Vehicle Body Types present in Accidents", \
    ylabel = "", autopct="%1.1f%%", pctdistance = 0.9, rotatelabels = False)

In [ ]:
# Driver was Drinking
vehDrinkDf = pd.DataFrame(crssDf["vehicle"]["VEH_ALCH"].value_counts().sort_values(ascending=False))
vehDrinkDf.columns = ["TotalVehicles"]
vehDrinkDf["VEH_ALCH"] = vehDrinkDf.index.values
vehDrinkDf["VEH_ALCH"] = vehDrinkDf.apply(lambda row: crssCategories["VEH_ALCH"][row["VEH_ALCH"]],axis=1)
vehDrinkDf.set_index("VEH_ALCH",inplace=True)
totalVeh = len(crssDf["vehicle"])
#vehDrinkDf.loc["Others"] = [totalVeh - vehDrinkDf["TotalVehicles"].sum()]
vehDrinkDf

In [ ]:
_ = vehDrinkDf["TotalVehicles"].plot.pie(legend = False, title = "Driver of Vehicle Reported Drinking", \
    ylabel = "", autopct="%1.1f%%", pctdistance = 0.7, rotatelabels = False)

In [ ]:
# simplify access to driver information from "person" table
driversDf = crssDf["person"][crssDf["person"]["PER_TYP"] == 1]
totalDrivers = len(driversDf)

In [ ]:
# Driver's Sex
drSexDf = pd.DataFrame(driversDf["SEX"].value_counts().sort_values(ascending=False))
drSexDf.columns = ["TotalDrivers"]
drSexDf["SEX"] = drSexDf.index.values
drSexDf["SEX"] = drSexDf.apply(lambda row: crssCategories["SEX"][row["SEX"]],axis=1)
drSexDf.set_index("SEX",inplace=True)
# Combine 'Reported as Unknown' and 'Not Reported'
drSexDf.loc["Unknown"] = drSexDf.loc["Reported as Unknown"] + drSexDf.loc["Not Reported"]
drSexDf.drop(index=["Reported as Unknown","Not Reported"],inplace=True)
drSexDf

In [ ]:
_ = drSexDf["TotalDrivers"].plot.pie(legend = False, title = "Sex of Driver of Vehicle", \
    ylabel = "", autopct="%1.1f%%", pctdistance = 0.7, rotatelabels = False)

In [ ]:
# Driver's Age
drAgeDf = pd.DataFrame(driversDf["AGE"].value_counts().sort_index())
drAgeDf.columns = ["TotalDrivers"]
drAgeDf["AGE"] = drAgeDf.index.values
#drAgeDf["SEX"] = drAgeDf.apply(lambda row: crssCategories["SEX"][row["SEX"]],axis=1)
#drAgeDf.set_index("AGE",inplace=True)
# Combine 'Reported as Unknown' and 'Not Reported'
drAgeDf.loc["Unknown"] = [drAgeDf.loc[998]["TotalDrivers"] + drAgeDf.loc[999]["TotalDrivers"],np.NaN]
drAgeDf.drop(index=[998,999],inplace=True)
drAgeDf

In [ ]:
drAgeDf["AGE"].describe()

In [ ]:
#_ = drAgeDf.drop(index="Unknown").plot.hist(by="TotalDrivers")
_ = driversDf[driversDf["AGE"] < 998]["AGE"].plot.hist(bins=20,title="Accident Driver Age Distribution")

In [ ]:
driverAgeCut = pd.cut(driversDf[driversDf["AGE"] < 998]["AGE"],[0,14,24,34,44,54,64,200],labels=["<15","15-24","25-34","35-44","45-54","55-64",">65"])
#_ = driverAgeCut.plot.scatter(title="Accident Driver Age Distribution")
#driverAgeCut.value_counts().plot.bar()
ax = driverAgeCut.value_counts(sort=False).plot.bar(title="Driver Age Distribution in 2020 Sampled Accidents",xlabel="Driver Age",ylabel="Number of accidents")

In [ ]:
fig = plt.figure(figsize=(10,5))
ax1 = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2)
driversDf[driversDf["AGE"] < 998].boxplot(column="AGE",ax=ax1)
ax1.set_title("Driver's Age Boxplot")
driversDf[(driversDf["AGE"] < 998) & (driversDf["SEX"].isin([1,2]))].boxplot(column="AGE",by="SEX",ax=ax2)
ax2.set_title("Driver's Age Boxplot by Sex")
a = ax2.get_xticklabels()
ax2.set_xticklabels([crssCategories["SEX"][int(a[0].get_text())],crssCategories["SEX"][int(a[1].get_text())]])
fig.suptitle("")


## Safety features

In [ ]:
safetyFeaturesColumns = [
 	"ForwardCollisionWarningId",
	"DynamicBrakeSupportId",
	"CrashImminentBrakingID",
	"PedestrianAutoEmergencyBrakingId",
	"BlindSpotWarningId",
	"BlindSpotInterventionId",
	"LaneDepartureWarningId",
	"LaneKeepingAssistanceId",
	"LaneCenteringAssistanceId",
	"BackupCameraId",
	"RearCrossTrafficAlertId",
	"RearAutomaticEmergencyBrakingId",
	"ParkAssistId",
	"DaytimeRunningLightId",
	#"HeadlampLightSourceId",
	"SemiAutoHeadlampBeamSwitchingId",
	"AdaptiveDrivingBeamId",
	"AdaptiveCruiseControlId",
	"AntilockBrakeSystemId",
	"ElectronicStabilityControlId",
	"TPMSId",
	"TractionControlId",
	"AutoPedestrianAlertingSoundId",
]

safetyFeaturesNames = {
 	"ForwardCollisionWarningId" : "Forward Collision Warning",
	"DynamicBrakeSupportId" : "Dynamic Brake Support",
	"CrashImminentBrakingID" : "Crash Imminent Braking",
	"PedestrianAutoEmergencyBrakingId" : "Pedestrian Auto Emergency Braking",
	"BlindSpotWarningId" : "Blind Spot Warning",
	"BlindSpotInterventionId": "Blind Spot Intervention",
	"LaneDepartureWarningId" : "Lane Departure Warning",
	"LaneKeepingAssistanceId" : "Lane Keeping Assistance",
	"LaneCenteringAssistanceId" : "Lane Centering Assistance",
	"BackupCameraId" : "Backup Camera",
	"RearCrossTrafficAlertId" : "Rear Cross Traffic Alert",
	"RearAutomaticEmergencyBrakingId" : "Rear Automatic Emergency Braking",
	"ParkAssistId" : "Park Assist",
	"DaytimeRunningLightId" : "Daytime Running Light",
	#"HeadlampLightSourceId",
	"SemiAutoHeadlampBeamSwitchingId" : "Semi Auto Headlamp Beam Switching",
	"AdaptiveDrivingBeamId" : "Adaptive Driving Beam",
	"AdaptiveCruiseControlId" : "Adaptive Cruise Control",
	"AntilockBrakeSystemId" : "Antilock Brake System",
	"ElectronicStabilityControlId" : "Electronic Stability Control",
	"TPMSId" : "Tire Pressure Monitoring System",
	"TractionControlId" : "Traction Control",
	"AutoPedestrianAlertingSoundId" : "Auto Pedestrian Alerting Sound",
}


def AnyStdSafetyFeature(vpicRow):
	"""Returns true if any of the safety features are marked standard for the vehicle checked 
	(vpicdecode columnnames are used)attribute value = 1 means standard
	"""
	
	stdFeatureFlag = False
	for feat in safetyFeaturesColumns:
		stdFeatureFlag = stdFeatureFlag | (vpicRow[feat] == 1)
	return stdFeatureFlag

def GoodVPICDecode(picDecodeResult):
	return (picDecodeResult in {'0','0,10','1,10','1,400','1,10,400'})


In [ ]:
crssDf["vpicdecode"].info(verbose=True,show_counts=True)

In [ ]:
crssDf["vpicdecode"].describe(include="all")

In [ ]:
crssDf["vpicdecode"]["VINDecodeError"].value_counts()

In [ ]:
badVPICcount = len(crssDf["vpicdecode"]) - (crssDf["vpicdecode"]["VINDecodeError"] == '0').sum()
print("Bad VIN Decodes = ",badVPICcount, "(", badVPICcount/len(crssDf["vpicdecode"])*100,"%)")

In [ ]:
crssDf["vpicdecode"]["AnySafetyFeature"] = crssDf["vpicdecode"].apply(lambda row: AnyStdSafetyFeature(row),axis=1)
#crssDf["vpicdecode"]["GoodVPICDecode"] = crssDf["vpicdecode"].apply(lambda row: (row["VINDecodeError"] == '0'),axis=1)
crssDf["vpicdecode"]["GoodVPICDecode"] = crssDf["vpicdecode"].apply(lambda row: (GoodVPICDecode(row["VINDecodeError"])),axis=1)


In [ ]:
print(crssDf["vpicdecode"]["GoodVPICDecode"].sum())
print(crssDf["vpicdecode"]["AnySafetyFeature"].sum())
badVPICcount = len(crssDf["vpicdecode"]) - crssDf["vpicdecode"]["GoodVPICDecode"].sum()
print("Bad VIN Decodes = ",badVPICcount, "(", badVPICcount/len(crssDf["vpicdecode"])*100,"%)")

In [ ]:
badVINrecords = crssDf["vpicdecode"][crssDf["vpicdecode"]["GoodVPICDecode"]==False]
badVINrecords

In [ ]:
print("Vehicles in vehicle table vs vpicdecode tables")
print("VPIC entries =",len(crssDf["vpicdecode"]),"Vehicle Entries =",len(crssDf["vehicle"]),"Vehicle Entires with Unkown Year =",(crssDf["vehicle"]["MOD_YEAR"] >= 9998).sum())

In [ ]:
# Vehicle Safety Features vs Model Year
# join vehicle table with vpictable 
#vehSafetyYrDf = pd.merge(crssDf["vehicle"][crssDf["vehicle"]["MOD_YEAR"] < 9998],crssDf["vpicdecode"], on = ["CASENUM","VEH_NO"])
vehVpicMergeDf = pd.merge(crssDf["vehicle"],crssDf["vpicdecode"], on = ["CASENUM","VEH_NO"])
vehSafetyYrDf = vehVpicMergeDf[["MOD_YEAR","AnySafetyFeature"]].groupby("MOD_YEAR").agg(['count','sum'])
vehSafetyYrDf = vehSafetyYrDf["AnySafetyFeature"]
vehSafetyYrDf.columns = ["Total Cars","Cars with Safey Features"]
#vehSafetyYrDf = pd.DataFrame(crssDf["vpicdecode"][["ModelYear","AnySafetyFeature"]].value_counts())
#vehSafetyYrDf.columns = ["TotalVehicles"]
#vehSafetyYrDf["VPICBODYCLASS"] = vehSafetyYrDf.index.values
#vehSafetyYrDf["VPICBODYCLASS"] = vehSafetyYrDf.apply(lambda row: crssCategories["VPICBODYCLASS"][row["VPICBODYCLASS"]],axis=1)
#vehSafetyYrDf.set_index("VPICBODYCLASS",inplace=True)
#vehSafetyYrDf["STATE"] = accStatesDf.apply(lambda row: crssCategories["STATE"][row["STATE"]],axis=1)
vehSafetyYrDf.info(verbose=True,show_counts=True)
#        ["MAKE","MAKENAME"],
#        ["MAK_MOD","MAK_MODNAME"],
##tempDf = vehSafetyYrDf[vehSafetyYrDf["MOD_YEAR"] != vehSafetyYrDf["ModelYear"]][["MOD_YEAR","ModelYear","MAKE","MAK_MOD","MakeId","ModelId"]].copy()
##temp2Df = vehSafetyYrDf[vehSafetyYrDf["MOD_YEAR"] == vehSafetyYrDf["ModelYear"]][["MOD_YEAR","ModelYear","MAKE","MAK_MOD","MakeId","ModelId"]].copy()
##makeMappingVehVpicDf = vehSafetyYrDf[["MAKE","MakeId"]].drop_duplicates().dropna()
##modelMappingVehVpicDf = vehSafetyYrDf[["MAKE","MAK_MOD","MakeId","ModelId"]].drop_duplicates().dropna()

##makeMappingVehVpicDf["MakeVpic"] = makeMappingVehVpicDf.apply(lambda row: crssCategories["MakeId"][row["MakeId"]],axis=1)
##makeMappingVehVpicDf["MAKENAME"] = makeMappingVehVpicDf.apply(lambda row: crssCategories["MAKE"][row["MAKE"]],axis=1)

##modelMappingVehVpicDf["MakeVpic"] = modelMappingVehVpicDf.apply(lambda row: crssCategories["MakeId"][row["MakeId"]],axis=1)
##modelMappingVehVpicDf["MAKENAME"] = modelMappingVehVpicDf.apply(lambda row: crssCategories["MAKE"][row["MAKE"]],axis=1)
##modelMappingVehVpicDf["ModelVpic"] = modelMappingVehVpicDf.apply(lambda row: crssCategories["ModelId"][row["ModelId"]],axis=1)
##modelMappingVehVpicDf["MAK_MODNAME"] = modelMappingVehVpicDf.apply(lambda row: crssCategories["MAK_MOD"][row["MAK_MOD"]],axis=1)


In [ ]:
vehSafetyFeatDf = vehVpicMergeDf[["AnySafetyFeature"]].value_counts().rename({True:"Yes", False:"No"})
vehSafetyFeatDf

In [ ]:
print(vehVpicMergeDf["MOD_YEAR"].drop_duplicates().sort_values().values)
print(crssDf["vehicle"]["MOD_YEAR"].drop_duplicates().sort_values().values)
print(crssDf["vpicdecode"]["ModelYear"].drop_duplicates().sort_values().values)

In [ ]:
fig = plt.figure(figsize=(20,5))
ax1 = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2)
vehSafetyYrDf.plot.bar(stacked=False,ax=ax1)
ax1.set_xlabel("Model Year")
ax1.set_ylabel("Number of Cars")
ax1.set_title("Total Cars vs Cars with Any Safety Features Grouped by model year for US 2020 Sampled Accidents")
vehVpicMergeDf[["AnySafetyFeature"]].value_counts().rename({True:"Yes", False:"No"}).plot.pie(legend = False, \
    ylabel = "", autopct="%1.1f%%", pctdistance = 0.7, rotatelabels = False, ax=ax2)
_ = ax2.set_title("Proportion of Vehicles with at least one Safety Feature")

In [ ]:
tempDf = vehVpicMergeDf[safetyFeaturesColumns].fillna(int(0))
vehSafetyFeaturesCnt = pd.Series(dtype='float64')
print("check values for every safety feature")
for x in tempDf.columns.sort_values():
    vehSafetyFeaturesCnt[safetyFeaturesNames[x]] = (tempDf[x] == 1).sum()/totalVeh*100
    print(x.ljust(max([len(x) for x in tempDf.columns])+2),"\t",str(tempDf[x].drop_duplicates().sort_values().values).ljust(16),(tempDf[x] == 1).sum())
tempDf = None
vehSafetyFeaturesCnt.sort_values(inplace=True)
vehSafetyFeaturesCnt

In [ ]:
ax = vehSafetyFeaturesCnt.plot.barh(title="Sampled Accidents: Percentage of Cars with the Safety Feature as Std",zorder=3)
ax.set_xlabel("Percentage of Vehicles")
ax.grid()